In [1]:
from os import system
import pickle

import numpy as np

## PFLOTRAN
import jupypft.model as mo
import jupypft.parameter as pm
import jupypft.plotBTC as plotBTC

def resetPickle():
    mo.Model.resetListOfModels()
    global caseDict
    with open('caseDict.pkl', 'rb') as f:
        caseDict = pickle.load(f)
    
    pm.Parameter.rebuildListOfObjects(caseDict)
    
    global listOfAllParameters
    listOfAllParameters = pm.Parameter.list_of_vars()
    
    system("rm -rf CASE*")
    
def plotResults(clean=True,BTC=False,EndC=False):
    if clean:
        system("rm -rf MASSBALANCES; mkdir MASSBALANCES")
        system("cp ./CASE**/*-mas.dat ./MASSBALANCES")
        mo.Model.folderFixedToCSV("MASSBALANCES")

    waterDensity = 999.65
    m3ToL = 1000.

    if BTC:
        plotBTC.plotMassBalancesInFolder(
            folderToPlot="MASSBALANCES",
            indices = {'t':"Time [d]",\
                       'q':"ExtractWell Water Mass [kg/d]",\
                       'm':"ExtractWell Vaq [mol/d]"},
            normalizeWith={'t':1.0,'q':waterDensity/m3ToL,'m':1.0},
            legendTitle = legendTitle)

    if EndC:
        plotBTC.plotEndConcentrations(
            folderToPlot="MASSBALANCES",
            Xdata = Iarray,
            indices = {'t':"Time [d]",\
                       'q':"ExtractWell Water Mass [kg/d]",\
                       'm':"ExtractWell Vaq [mol/d]"},
            normalizeWith={'t':1.0,'q':waterDensity/m3ToL,'m':1.0},
            legendTitle = legendTitle)
    
def buildSim(caseName):
    ## Create a folder for the case
    currentFolder = "./CASE_{0}".format(caseName)
    currentFile = currentFolder + "/" + caseName +".in"
    system("mkdir " + currentFolder)
    
    ## Initialize PFLOTRAN model
    BoxModel = mo.Model(
        templateFile = templateFile,
        runFile = currentFile,
        execPath = execPath,
        verbose=True
        )
       
    ## Copy template input file to folder
    BoxModel.cloneTemplate()
    
    ## Replace tags for values in case
    for parameter in listOfAllParameters:
        BoxModel.replaceTagInFile(parameter)

    return BoxModel

## Dummy for caseDict
caseDict = {}

def getTemplate(key):
    templateFiles = {"TH_RSandbox_Const":"tpl_TH_3Dbox_bioparticleKte_NoH5.in"}

    templateFolder = "../TEMPLATES/boxes_3D/"
    return templateFolder + templateFiles[key]
    
templateFile = getTemplate('TH_RSandbox_Const')
execPath = "$PFLOTRAN_DIR/buildExperimental/pflotran"

In [2]:
K = 10.**-2.
Qin = 0.24
f = 10.
H = 20.
r = 40.
I = 0.001
C0 = 1.0
decayRate = 3.5353E-06

nu = 0.0000013081 #m²/s
g = 9.81 #m/s²
THETA = 0.35

In [3]:
resetPickle()
caseDict['endTime'].value = 50.

caseDict['BIOPARTICLE']['decayAq'].value = decayRate
caseDict['Q']['In'].value  = Qin
caseDict['Q']['Out'].value = -Qin*f
caseDict['k']['X'].value = K*nu/g
caseDict['k']['Y'].value = K*nu/g
caseDict['k']['Z'].value = K*nu/g
caseDict['theta'].value = THETA
caseDict['L']['Z'].value = H
caseDict['inCoord']['X'][1].value = caseDict['outCoord']['X'][1].value + r
caseDict['inCoord']['X'][2].value = caseDict['outCoord']['X'][2].value + r

caseDict['inCoord']['Z'][1].value = 0.0
caseDict['inCoord']['Z'][2].value = H

caseDict['outCoord']['Z'][1].value = 0.0
caseDict['outCoord']['Z'][2].value = H

FileNotFoundError: [Errno 2] No such file or directory: 'caseDict.pkl'